In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 1. Load data

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
print('train: ')
display(train_df.head(1))
print('test: ')
display(test_df.head(1))

train: 


,id,0,1,2,3,4,5,6,7,8,...,178,179,180,181,182,183,184,185,186,187
0,1,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


test: 


,id,0,1,2,3,4,5,6,7,8,...,177,178,179,180,181,182,183,184,185,186
0,1,1.0,0.805419,0.206897,0.135468,0.182266,0.20936,0.204434,0.14532,0.179803,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X = train_df.iloc[:, 1: -1]
y = train_df.iloc[:, -1]
pd.value_counts(y)

0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: 187, dtype: int64

import pandas as

In [1]:
import numpy as np
import pandas as pd

In [4]:
preds = []
for i in range(5774):
    if i < 2007:
        preds.append(0)
    elif i < 2556:
        preds.append(1)
    elif i < 4009:
        preds.append(2)
    elif i < 4166:
        preds.append(3)
    else:
        preds.append(4)
preds = np.asarray(preds)
results = pd.DataFrame()
results['id'] = [i for i in range(1, preds.shape[0]+1)]       
results['predicted'] = preds      
results.to_csv('sample_sub.csv', index=False)

## Logits

In [52]:
import os.path as path
from sklearn.metrics import confusion_matrix
import plotly.express as px
def get_cm(pred, target, normalize='true'):
    cm = confusion_matrix(target, pred, normalize=normalize)
    cm = np.around(cm, 4)
    fig = px.imshow(cm,
                x=['N', 'S', 'V', 'N+V', 'U'],
                y=['N', 'S', 'V', 'N+V', 'U'],
                text_auto=True, aspect="auto"
        )
    fig.update_xaxes(side="top")
    fig.show()
    
def output_pred(preds, output_path):
    results = pd.DataFrame()
    results['id'] = [i for i in range(1, preds.shape[0]+1)]       
    results['predicted'] = preds.astype(int)      
    results.to_csv(output_path, index=False)

In [59]:
log_dir = './logs/network=LSTM_d-hidden=256_n-layers=1_weighted_loss=True_n-freqs=4/'
i_epoch = 1000
logits_path = path.join(log_dir, f'logits_{i_epoch}.npy')
# logits, target = np.load(logits_path).split([5, 1], axis=1)
# logits, target = np.split(np.load(logits_path), [5], axis=1)
# logits.shape, target.shape
logits = np.load(logits_path)
preds = np.argmax(logits[:, :5], axis=1).astype(float)
logits = pd.DataFrame(logits, columns=['N', 'S', 'V', 'N+V', 'U', 'target'])
logits['pred'] = preds
logits['modified_pred'] = preds
logits.head(2), logits.shape

(     N             S             V           N+V             U  target  pred  \
 0  1.0  1.904247e-12  6.219869e-12  1.030566e-10  1.974013e-08     0.0   0.0   
 1  1.0  1.416530e-14  7.685113e-20  1.246678e-14  1.437435e-14     0.0   0.0   
 
    modified_pred  
 0            0.0  
 1            0.0  ,
 (4378, 8))

In [55]:
get_cm(logits['pred'], logits['target'])

In [56]:
logits_1 = logits[logits['target']==1]
logits_1_FP = logits_1[logits_1['pred']!=1]
print(logits_1.shape, logits_1_FP.shape)

(104, 8) (17, 8)


In [57]:
logits.loc[(logits['S']>0.05) & (logits['pred']==0), 'modified_pred'] = 1
logits.loc[(logits['V']>0.05) & (logits['pred']==0), 'modified_pred'] = 2
logits.loc[(logits['N+V']>0.05) & (logits['pred']==0), 'modified_pred'] = 3
get_cm(logits['modified_pred'], logits['target'])

In [58]:
# output_pred(logits['modified_pred'], path.join(log_dir, f'mod_output_{i_epoch}.csv'))
logits.shape

(4378, 8)